In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Post-Traumatic_Stress_Disorder"
cohort = "GSE52875"

# Input paths
in_trait_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder"
in_cohort_dir = "../../input/GEO/Post-Traumatic_Stress_Disorder/GSE52875"

# Output paths
out_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/GSE52875.csv"
out_gene_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/gene_data/GSE52875.csv"
out_clinical_data_file = "../../output/preprocess/Post-Traumatic_Stress_Disorder/clinical_data/GSE52875.csv"
json_path = "../../output/preprocess/Post-Traumatic_Stress_Disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression signatures in heart tissues of mice simulating posttraumatic stress disorder (PTSD)"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['strain: C57BL/6'], 1: ['tissue: Control_heart_tissue, C10R42', 'tissue: Control_heart_tissue, C10R1', 'tissue: Stressed_heart_tissue, T10R1', 'tissue: Control_heart_tissue, C5R1', 'tissue: Control_heart_tissue, C5R10', 'tissue: Stressed_heart_tissue, T10R42', 'tissue: Stressed_heart_tissue, T5R10', 'tissue: Stressed_heart_tissue, T5R1']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine Gene Expression Data Availability
# Based on the background info and sample characteristics, this dataset appears to be related to gene expression 
# in heart tissues of mice simulating PTSD. The title suggests it's an expression study.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Looking at the sample characteristics dictionary, we don't see explicit trait (PTSD),
# age, or gender information. The dictionary only contains strain and tissue information.
trait_row = None  # No explicit PTSD status in the sample characteristics
age_row = None    # No age information available
gender_row = None # No gender information available

# 2.2 Data Type Conversion Functions
# Since we don't have these variables in the data, we'll define placeholder functions
# that would be appropriate if the data were available
def convert_trait(value):
    """
    Convert PTSD status to binary format.
    Expected format: "status: value"
    """
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower()
    
    if value in ['ptsd', 'yes', 'true', 'case', 'positive', '1']:
        return 1
    elif value in ['control', 'no', 'false', 'normal', 'negative', '0']:
        return 0
    return None

def convert_age(value):
    """
    Convert age to continuous format.
    Expected format: "age: value"
    """
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """
    Convert gender to binary format (0=female, 1=male).
    Expected format: "gender: value" or "sex: value"
    """
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower()
    
    if value in ['male', 'm', 'man', 'boy', '1']:
        return 1
    elif value in ['female', 'f', 'woman', 'girl', '0']:
        return 0
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save the initial filtering metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip the clinical feature extraction
# The dataset appears to be a mouse model study without explicit trait classification in the sample characteristics


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['-1', '10138', '10306', '105441', '10899', '10901', '10902', '10903',
       '10904', '10905', '10906', '10907', '10916', '10919', '10923', '10925',
       '10928', '10936', '10937', '10942'],
      dtype='object', name='ID')

Gene data dimensions: 2025 genes × 32 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be simple numeric values (1, 2, 3, etc.)
# These are not standard human gene symbols like BRCA1, TP53, etc.
# These are likely probe IDs or some other identifier that needs to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['10916', '10998', '42918', '17883', '10997'], 'name': ['hsa-miR-1/mmu-miR-1', 'hsa-miR-19b/mmu-miR-19b/rno-miR-19b', 'hsa-miR-19b-2*', 'hsa-miR-19b-1*', 'hsa-miR-19a/mmu-miR-19a/rno-miR-19a'], 'accession': ['MIMAT0000416/MIMAT0000123', 'MIMAT0000074/MIMAT0000513/MIMAT0000788', 'MIMAT0004492', 'MIMAT0004491', 'MIMAT0000073/MIMAT0000651/MIMAT0000789'], 'miRNA_LIST': ['hsa-miR-1/mmu-miR-1', 'hsa-miR-19b/mmu-miR-19b/rno-miR-19b', 'hsa-miR-19b-2*', 'hsa-miR-19b-1*', 'hsa-miR-19a/mmu-miR-19a/rno-miR-19a'], 'SEQUENCE': ['UGGAAUGUAAAGAAGUAUGUAU', 'UGUGCAAAUCCAUGCAAAACUGA', 'AGUUUUGCAGGUUUGCAUUUCA', 'AGUUUUGCAGGUUUGCAUCCAGC', 'UGUGCAAAUCUAUGCAAAACUGA'], 'database': ['miRBase 14.0', 'miRBase 14.0', 'miRBase 14.0', 'miRBase 14.0', 'miRBase 14.0'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation preview, this dataset contains miRNA data, not gene expression data
# The "ID" column does match the numeric identifiers in the gene expression data
# However, miRNA_LIST contains miRNA identifiers, not human gene symbols
print("Column names in gene_annotation:", gene_annotation.columns.tolist())
print("\nThis appears to be a miRNA dataset, not gene expression data")

# This is a critical issue as our pipeline requires gene expression data that can be mapped to human gene symbols
# According to the background info, this is a mouse study with miRNA data

# Update our assessment of gene data availability
is_gene_available = False  # miRNA data is not suitable for our gene expression analysis pipeline

# Save the updated metadata reflecting that this dataset doesn't contain usable gene expression data
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

print("\nDataset rejected: This dataset contains miRNA data from mice rather than human gene expression data.")

Column names in gene_annotation: ['ID', 'name', 'accession', 'miRNA_LIST', 'SEQUENCE', 'database', 'SPOT_ID']

This appears to be a miRNA dataset, not gene expression data

Dataset rejected: This dataset contains miRNA data from mice rather than human gene expression data.
